In [7]:
import random
from random import shuffle

from PIL import Image, ImageFilter
from IPython import display

from matplotlib.pyplot import imshow
%matplotlib inline

import pickle
import time
import error_table

from sampling_utils import *
from gen_utils import *
from config_db import ConfigDB

In [8]:
num_pics = 10
min_num_cars = 1
max_num_cars = 4

domains = [[52,87],[1,37],[0,1],[0.35,1],[0.5,1.5],[0.25,1],[0.5,1],[0.5,1]]
types = ['int','int','float','float','float','float','float','float']

gen_misclass = True
store_config_db = False
store_err_tab = False

IOMIN_THRESH = 0.4
IOU_THRESH = 0.4

if gen_misclass:
    checkpoint = '/home/tommaso/analyzeNN/data/train_0/checkpoint/train/model.ckpt-5000'
    import squeezedet as nn
    net = nn.init(checkpoint)

INFO:tensorflow:Summary name mean iou is illegal; using mean_iou instead.
INFO:tensorflow:Restoring parameters from /home/tommaso/analyzeNN/data/train_0/checkpoint/train/model.ckpt-5000


In [9]:
for j in range(min_num_cars,max_num_cars):
    
    total_attempts = 0
    rejections = 0
    
    # Init data structure
    if store_err_tab:
        err_table = error_table()
    if store_config_db:
        config_db = ConfigDB()
    
    path_data_set = './data/test/'
    #path_data_set = './data/train_0/' + str(j) + '/'
    #path_data_set = '/home/tommaso/analyzeNN/data/train_0/mis/mis_' + str(j) + '/'
    
    i = 0
    v = 0

    while i < num_pics:

        # Sample configuration
        #num_cars = random.randint(min_num_cars, max_num_cars)
        num_cars = j
        sample_dims = [1,num_cars,num_cars,num_cars,1,1,1,1]
        sample, norm_sample = get_sample(v, 'random', types=types, domains=domains, dims=sample_dims)
        sample[3].sort(reverse=True)
        norm_sample[3].sort(reverse=True)
        
        total_attempts += 1
        
        good_sample = True
        # Check sample distance from stored ones
        if store_config_db:
            sample_pad = pad_sample(sample)
            good_sample = config_db.eps_dist(sample_pad,0.5)
            
            
        if good_sample:
            # Generate image from sample
            img, ground_boxes, img_info = gen_image(sample)

            # Is it a good picture? (cars don't overlap too much)
            save_img = True
            for m in range(len(ground_boxes)):
                for n in range(m+1, len(ground_boxes)):
                    #print(iomin(ground_boxes[m], ground_boxes[n]))
                    if iomin(ground_boxes[m], ground_boxes[n]) > IOMIN_THRESH:
                        save_img = False
                        
            flag = "g"

            # Check misclassification
            if gen_misclass and save_img:
                # Save tmp image    
                tmp_img_file_name = './tmp.png'
                img.save(tmp_img_file_name)

                # Classify img
                (boxes,probs,cats) = nn.classify(tmp_img_file_name,net) 

                save_img = False

                # Not cars
                if any(cats):
                    save_img = True
                else:
#                     all_good_boxes = True
#                     for ground_box in ground_boxes:
#                         found_good_box = False
#                         for box, prob in zip(boxes, probs):    
#                             found_good_box = found_good_box or ((iou(ground_box, box) > IOU_THRESH) and prob > 0.5)
#                         all_good_boxes = all_good_boxes and found_good_box
#                     save_img = not(all_good_boxes)
                    prec, rec = precision_recall(boxes, ground_boxes,IOU_THRESH)
                    if prec <= 0.75 or rec <= 0.75:
                        save_img = True
                        

            if save_img:
                
                if store_config_db:
                    config_db.add(sample_pad)            
                
                file_name = 'h_m_0_' + str(j) + '_' + str(i).zfill(6)
                #file_name = 'm_0_' + str(j) + '_' + str(i).zfill(6)
                #file_name = str(j) + '_' + str(i).zfill(6)

                # Save data
                save_image(img, file_name, path_data_set)
                save_label(ground_boxes, file_name, path_data_set)
                
                if store_err_tab:
                    err_table.update_with_elem(img_info)
                    
                i+=1
        else:
            rejections += 1

        if(total_attempts %50 == 0):
            print('gen pics: ' + str(total_attempts) + ' | misclass pics: ' + str(i) + ' | rejections: ' +str(rejections))
            
    print('END gen pics: ' + str(total_attempts) + ' | misclass pics: ' + str(i) + ' | rejections: ' +str(rejections))
            
    if store_err_tab:
        err_table_file = open(path_data_set + 'error_table' + str(),'w')
        pickle.dump(err_table,err_table_file)
        err_table_file.close()

END gen pics: 47 | misclass pics: 10 | rejections: 0
gen pics: 50 | misclass pics: 2 | rejections: 0
gen pics: 100 | misclass pics: 3 | rejections: 0
gen pics: 150 | misclass pics: 5 | rejections: 0
gen pics: 200 | misclass pics: 7 | rejections: 0
END gen pics: 232 | misclass pics: 10 | rejections: 0
gen pics: 50 | misclass pics: 0 | rejections: 0
gen pics: 100 | misclass pics: 0 | rejections: 0
gen pics: 150 | misclass pics: 0 | rejections: 0
gen pics: 200 | misclass pics: 0 | rejections: 0
gen pics: 250 | misclass pics: 0 | rejections: 0
gen pics: 300 | misclass pics: 0 | rejections: 0
gen pics: 350 | misclass pics: 0 | rejections: 0
gen pics: 400 | misclass pics: 0 | rejections: 0
gen pics: 450 | misclass pics: 0 | rejections: 0
gen pics: 500 | misclass pics: 1 | rejections: 0
gen pics: 550 | misclass pics: 1 | rejections: 0
gen pics: 600 | misclass pics: 1 | rejections: 0
gen pics: 650 | misclass pics: 1 | rejections: 0
gen pics: 700 | misclass pics: 1 | rejections: 0
gen pics: 750